# **ANALISIS DE PRATRONES**
Para poder identificar los patrones, vamos a realizar un dataframes con el top50 de vivientas ocupadas, y el top50 de las peores viviendas en cuanto a alquiler. 

In [2]:
# Importamos las librerias necesarias
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
from plotly.subplots import make_subplots
import zipfile
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import scipy.stats as stats
from scipy.stats import skew
from scipy.stats import f_oneway
import folium
from folium.plugins import FastMarkerCluster
import scipy.stats as stats


In [3]:
# Mostrar todas las columnas y filas de nuestro dataframes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Configuración del formato de numeros
pd.options.display.float_format = '{:.2f}'.format

In [4]:
# Leemos dataframes
df_listings = pd.read_csv(r'/Users/juliobrionesmorales/Documents/GitHub/Airbnb_Barcelona/data/3. data_sin_outliers/df_listings.csv')
df_reviews = pd.read_csv(r'/Users/juliobrionesmorales/Documents/GitHub/Airbnb_Barcelona/data/3. data_sin_outliers/df_reviews')
df_calendar = pd.read_csv(r'/Users/juliobrionesmorales/Documents/GitHub/Airbnb_Barcelona/data/3. data_sin_outliers/df_calendar')

In [82]:
# Mostramos el head de listings
df_listings.head()

,id,listing_url,name,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,latitude,longitude,price,minimum_nights,maximum_nights,accommodates,bedrooms,beds,bathrooms_text,amenities,number_of_reviews,number_of_reviews_ltm,reviews_per_month,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_value,host_is_superhost,host_response_rate,host_listings_count,availability_365,calculated_host_listings_count
0,360863.00,https://www.airbnb.com/rooms/360863,SPECIAL OFFER @ COSY AND CHARMING,la Barceloneta,Ciutat Vella,Private room in rental unit,Private room,41.38,2.20,173.00,31.00,150.00,2.00,2.18,2.73,1 private bath,"[""Cleaning available during stay"", ""Baking she...",32.00,0.00,0.22,4.73,4.84,4.94,4.71,4.74,t,6%,2.00,0.00,2.00
1,17475.00,https://www.airbnb.com/rooms/17475,Attic Sagrada Familia,la Dreta de l'Eixample,Eixample,Entire rental unit,Entire home/apt,41.40,2.17,165.00,3.00,90.00,3.00,1.00,1.00,1 bath,"[""Microwave"", ""Outdoor dining area"", ""Private ...",29.00,12.00,0.17,4.48,4.23,4.73,4.82,4.50,f,100%,2.00,7.00,1.00
2,18674.00,https://www.airbnb.com/rooms/18674,Huge flat for 8 people close to Sagrada Familia,la Sagrada Família,Eixample,Entire rental unit,Entire home/apt,41.41,2.17,210.00,1.00,1125.00,7.00,3.00,6.00,2 baths,"[""Paid parking on premises"", ""Private patio or...",40.00,6.00,0.30,4.33,4.62,4.67,4.77,4.28,f,97%,44.00,252.00,28.00
3,97529.00,https://www.airbnb.com/rooms/97529,Furnished one bedroom apartment with bathroom,el Camp d'en Grassot i Gràcia Nova,Gràcia,Entire rental unit,Entire home/apt,41.41,2.17,170.00,76.00,1125.00,2.00,1.00,4.27,1 bath,"[""Microwave"", ""Paid parking off premises"", ""Pr...",67.00,0.00,0.45,4.74,4.86,4.91,4.45,4.71,f,20%,1.00,0.00,1.00
4,110041.00,https://www.airbnb.com/rooms/110041,002. Alió 2 Apartment,el Camp d'en Grassot i Gràcia Nova,Gràcia,Entire rental unit,Entire home/apt,41.40,2.17,94.00,1.00,1125.00,4.00,1.00,2.00,1 bath,"[""Microwave"", ""Iron"", ""Dedicated workspace"", ""...",54.00,18.00,0.38,4.17,4.24,4.41,4.65,4.11,f,100%,97.00,226.00,78.50


In [83]:
df_listings_copy = df_listings.copy()

In [84]:
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18519 entries, 0 to 18518
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              18519 non-null  float64
 1   listing_url                     18519 non-null  object 
 2   name                            18519 non-null  object 
 3   neighbourhood_cleansed          18519 non-null  object 
 4   neighbourhood_group_cleansed    18519 non-null  object 
 5   property_type                   18519 non-null  object 
 6   room_type                       18519 non-null  object 
 7   latitude                        18519 non-null  float64
 8   longitude                       18519 non-null  float64
 9   price                           18519 non-null  float64
 10  minimum_nights                  18519 non-null  float64
 11  maximum_nights                  18519 non-null  float64
 12  accommodates                    

In [85]:
# Antes de comenzar conla correlación, vamos a eliminar elementos que no nos interesen para nuestro analisis
df_listings.drop(columns=['name','amenities','calculated_host_listings_count','bathrooms_text','listing_url'], inplace=True)

In [86]:
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18519 entries, 0 to 18518
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            18519 non-null  float64
 1   neighbourhood_cleansed        18519 non-null  object 
 2   neighbourhood_group_cleansed  18519 non-null  object 
 3   property_type                 18519 non-null  object 
 4   room_type                     18519 non-null  object 
 5   latitude                      18519 non-null  float64
 6   longitude                     18519 non-null  float64
 7   price                         18519 non-null  float64
 8   minimum_nights                18519 non-null  float64
 9   maximum_nights                18519 non-null  float64
 10  accommodates                  18519 non-null  float64
 11  bedrooms                      18519 non-null  float64
 12  beds                          18519 non-null  float64
 13  n

In [87]:
df_listings.columns

Index(['id', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed',
       'property_type', 'room_type', 'latitude', 'longitude', 'price',
       'minimum_nights', 'maximum_nights', 'accommodates', 'bedrooms', 'beds',
       'number_of_reviews', 'number_of_reviews_ltm', 'reviews_per_month',
       'review_scores_rating', 'review_scores_cleanliness',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'host_is_superhost', 'host_response_rate',
       'host_listings_count', 'availability_365'],
      dtype='object')

In [88]:
df_listings['availability_365'].unique()

array([  0.,   7., 252., 226., 210., 198., 166., 204., 250., 277., 248.,
        47., 208., 238., 242., 339.,  93., 365., 224., 282., 326., 191.,
       291., 316.,  88., 148., 114.,  33.,  58., 165., 201., 185., 293.,
       327., 335., 309., 364., 228.,  83., 305., 125., 116., 167., 267.,
       153., 197., 296., 163.,  54., 123., 363.,  78., 189., 278.,  12.,
         1., 348., 157., 179., 258., 362.,  76., 229., 181., 227., 184.,
       353.,  57.,  91., 355., 262., 140., 294., 303.,  84., 283., 221.,
       297., 243., 245., 317., 255., 194.,  38., 193.,  11., 236.,  99.,
       334.,  45., 269.,  30., 112., 110., 129.,  52., 320., 338., 247.,
       275., 124., 101., 332., 130.,  75.,  60., 300., 103., 285.,  59.,
       288.,  79., 233., 235.,  87., 115., 276.,  82., 214.,   4., 292.,
       284., 211.,  77., 187., 241., 270., 351.,  25., 151., 230., 202.,
         5.,   3., 310., 225., 168., 280., 257., 144., 265., 220., 219.,
       212., 289., 260.,  23., 178.,  53., 100., 26

## CORRELACION DE VARIABLES
En este apartado vamos a determinar las variables que tienen más correlación con 'availability_365' con respecto a las otras variables para poder determinar luego el top50 de propiedades. También los vamos hacer con 'price'.

In [89]:
# CORRELACION PRICE

# Seleccionar solo las características numéricas
numeric_features = df_listings.select_dtypes(include=['float64'])

# Calcular la correlación con respecto a "price"
correlation_with_price = numeric_features.corrwith(df_listings['price'])

# Ordenar las correlaciones en orden descendente
sorted_correlation_with_price = correlation_with_price.abs().sort_values(ascending=False)

# Imprimir las correlaciones con respecto a "price" ordenadas de mayor a menor
print(sorted_correlation_with_price)

price                         1.00
beds                          0.57
accommodates                  0.55
bedrooms                      0.46
minimum_nights                0.40
number_of_reviews             0.25
id                            0.23
number_of_reviews_ltm         0.22
maximum_nights                0.11
review_scores_cleanliness     0.10
review_scores_location        0.10
review_scores_value           0.08
review_scores_communication   0.07
review_scores_rating          0.07
availability_365              0.05
reviews_per_month             0.04
longitude                     0.03
host_listings_count           0.01
latitude                      0.01
dtype: float64


In [90]:
# CORRELACION AVAILABILITY_365  

# Seleccionar solo las características numéricas
numeric_features = df_listings.select_dtypes(include=['float64'])

# Calcular la correlación con respecto a "price"
correlation_with_price = numeric_features.corrwith(df_listings['availability_365'])

# Ordenar las correlaciones en orden descendente
sorted_correlation_with_price = correlation_with_price.abs().sort_values(ascending=False)

# Imprimir las correlaciones con respecto a "price" ordenadas de mayor a menor
print(sorted_correlation_with_price)


availability_365              1.00
host_listings_count           0.27
minimum_nights                0.18
review_scores_value           0.18
id                            0.16
review_scores_communication   0.14
accommodates                  0.13
review_scores_rating          0.11
review_scores_cleanliness     0.08
review_scores_location        0.07
price                         0.05
beds                          0.04
longitude                     0.03
maximum_nights                0.03
number_of_reviews             0.03
bedrooms                      0.02
number_of_reviews_ltm         0.01
latitude                      0.01
reviews_per_month             0.00
dtype: float64


## TOP50 VIVIENDAS - MEJORES / PEORES

Para poder hacer las top200 mejores y peores, vamos a tener la correlaciones calculadas antes respecto al precio y respecto a la disponibilidad, para establecerlas en el weight de nuestro top200:

**AVAILABILITY_365**
- availability_365:              1.00
- host_listings_count:           0.27
- minimum_nights:                0.18
- review_scores_value:           0.18
- id:                            0.16
- review_scores_communication:   0.14
- accommodates:                  0.13
- review_scores_rating:          0.11
- review_scores_cleanliness:     0.08
- review_scores_location:        0.07
- price:                         0.05
- beds:                          0.04


**PRICE**
- price:                         1.00
- beds:                          0.57
- accommodates:                  0.55
- bedrooms:                      0.46
- minimum_nights:                0.40
- number_of_reviews:             0.25
- id:                            0.23
- number_of_reviews_ltm:         0.22
- maximum_nights:                0.11
- review_scores_cleanliness:     0.10
- review_scores_location:        0.10
- review_scores_value:           0.08


Ahora para hacer el top50 vamos a tener en cuenta en weights las varibles la correlacionadas. 

In [91]:
# Eliminamos duplicados basados en el ID de la vivienda
df_listings_drop_duplicates = df_listings.drop_duplicates(subset='id') 

# Ponderación de cada característica
weights = {
    'host_listings_count':0.2,
    'beds':0.2,
    'accommodates': 0.2,
    'minimum_nights': 0.1,
    'review_scores_value':0.1,
    'bedrooms': 0.1,
    'review_scores_communication': 0.05,
    'price': 0.05
}


# Nos aseguramos que no hay valores nulos
df_listings_drop_duplicates.fillna(0, inplace=True)

# Normalizar el precio para que esté entre 0 y 1
scaler = MinMaxScaler()
df_listings_drop_duplicates['price_normalized'] = scaler.fit_transform(df_listings_drop_duplicates[['price']])
df_listings_drop_duplicates['price_normalized']= 1-df_listings_drop_duplicates['price_normalized']

# Calculamos el puntaje ponderado
df_listings_drop_duplicates['weighted_score'] = (
    df_listings_drop_duplicates['host_listings_count'] * weights['host_listings_count'] +
    df_listings_drop_duplicates['beds'] * weights['beds'] +
    df_listings_drop_duplicates['accommodates'] * weights['accommodates'] +
    df_listings_drop_duplicates['minimum_nights'] * weights['minimum_nights'] +
    df_listings_drop_duplicates['review_scores_value'] * weights['review_scores_value'] +
    df_listings_drop_duplicates['bedrooms'] * weights['bedrooms'] +
    df_listings_drop_duplicates['review_scores_communication'] * weights['review_scores_communication'] +
    df_listings_drop_duplicates['price_normalized'] * weights['price']
)

# Ordenar por el puntaje ponderado para obtener las 50 mejores
top_200 = df_listings_drop_duplicates.sort_values(by='weighted_score', ascending=False).head(200)

# Mostrar los resultados de las 50 mejores viviendas
print(top_200[['id', 'beds', 'bedrooms', 'price', 'accommodates', 'weighted_score']])

# Ordenar por el puntaje ponderado para obtener las 50 peores
bottom_200 = df_listings_drop_duplicates.sort_values(by='weighted_score', ascending=True).head(200)

# Mostrar los resultados de las 50 peores viviendas
print(bottom_200[['id', 'beds', 'bedrooms', 'price', 'accommodates', 'weighted_score']])

                          id  beds  bedrooms  price  accommodates  \
16063  968720450586425984.00  4.00      3.00 248.00          6.00   
9444             50861575.00  3.00      3.00 122.00          6.00   
9603             51759096.00  3.00      3.00 148.00          6.00   
12395  744806258021033088.00  3.00      3.00 208.00          4.00   
9915             52933728.00  2.82      1.00 159.27          2.00   
7214             39571615.00  1.00      0.00  50.00          2.00   
7198             39548516.00  1.00      0.00  50.00          2.00   
6829             36824913.00  1.00      0.00  50.00          2.00   
10817  604823244440152064.00  3.00      3.00  78.00          6.00   
3134             16475404.00  2.91      3.00 184.82          4.00   
5780             30223189.00  1.00      1.00  81.00          2.00   
8136             42947434.00  6.00      3.50 125.00          6.00   
8139             42947823.00  6.00      3.50 110.00          6.00   
6825             36820320.00  6.00

In [92]:
# Mostramos las 50 mejores
top_200[['id','beds', 'bedrooms', 'price', 'accommodates', 'weighted_score']]

,id,beds,bedrooms,price,accommodates,weighted_score
16063,968720450586425984.00,4.00,3.00,248.00,6.00,29.97
9444,50861575.00,3.00,3.00,122.00,6.00,29.88
9603,51759096.00,3.00,3.00,148.00,6.00,29.77
12395,744806258021033088.00,3.00,3.00,208.00,4.00,29.41
9915,52933728.00,2.82,1.00,159.27,2.00,28.78
7214,39571615.00,1.00,0.00,50.00,2.00,28.39
7198,39548516.00,1.00,0.00,50.00,2.00,28.32
6829,36824913.00,1.00,0.00,50.00,2.00,28.32
10817,604823244440152064.00,3.00,3.00,78.00,6.00,28.15
3134,16475404.00,2.91,3.00,184.82,4.00,27.82


In [93]:
# Mostramos las 50 peores
bottom_200[['id', 'beds', 'bedrooms', 'price', 'accommodates', 'weighted_score']]

,id,beds,bedrooms,price,accommodates,weighted_score
16735,1016449343214723584.00,0.00,1.00,55.00,1.00,1.32
14642,900907375525248000.00,0.00,1.00,60.00,1.00,1.33
5844,31884524.00,0.00,1.00,45.00,1.00,1.34
14457,893890275442034048.00,0.00,1.00,70.00,1.00,1.34
12710,763883578421275520.00,0.00,1.00,65.00,1.00,1.36
13575,842126611333474048.00,0.00,0.00,70.00,1.00,1.36
16863,1023272532720098816.00,0.00,1.00,42.00,1.00,1.36
14858,914559057079443968.00,0.00,0.00,35.00,1.00,1.36
12853,785547170328341376.00,0.00,1.00,30.00,1.00,1.36
17202,1037001392007124736.00,0.00,1.00,43.00,1.00,1.37


In [94]:
# Realizamos un describe para poder analizar cada una de las variables numéricas
top_200.describe().T

,count,mean,std,min,25%,50%,75%,max
id,200.00,612849792337660032.00,425738999219372608.00,290535.00,53077512.00,782064730332971008.00,973853473693172224.00,1108153097338524800.00
latitude,200.00,41.39,0.01,41.37,41.38,41.39,41.40,41.43
longitude,200.00,2.16,0.02,2.12,2.15,2.16,2.17,2.22
price,200.00,144.26,51.27,26.00,113.61,138.00,161.45,341.39
minimum_nights,200.00,33.62,9.38,25.00,31.00,31.00,32.00,76.00
maximum_nights,200.00,515.25,321.25,180.00,330.00,365.00,365.00,1125.00
accommodates,200.00,5.17,1.07,1.00,5.00,5.00,6.00,7.00
bedrooms,200.00,2.90,0.60,0.00,3.00,3.00,3.00,3.50
beds,200.00,3.48,0.99,1.00,3.00,3.00,4.00,6.00
number_of_reviews,200.00,0.99,1.93,0.00,0.00,0.00,1.00,12.00


In [95]:
bottom_200.describe().T

,count,mean,std,min,25%,50%,75%,max
id,200.00,687503303564211200.00,394883843024895488.00,1604559.00,617907621096975232.00,840273759416531712.00,981892837931718144.00,1115157066470587136.00
latitude,200.00,41.40,0.02,41.37,41.38,41.40,41.41,41.45
longitude,200.00,2.17,0.02,2.09,2.15,2.17,2.18,2.22
price,200.00,72.10,41.80,27.00,45.00,60.00,80.00,264.00
minimum_nights,200.00,1.29,0.52,1.00,1.00,1.00,2.00,4.00
maximum_nights,200.00,408.42,393.37,1.00,31.00,365.00,365.00,1125.00
accommodates,200.00,1.30,0.47,1.00,1.00,1.00,2.00,3.00
bedrooms,200.00,0.95,0.29,0.00,1.00,1.00,1.00,2.00
beds,200.00,0.79,0.45,0.00,1.00,1.00,1.00,1.91
number_of_reviews,200.00,22.39,31.73,0.00,1.00,9.50,26.25,115.00


## COMPARACION GRAFICAS DE MEJORE Y PEORES PROPIEDADES

In [96]:
# Primero vemos las columnas que podemos analizar del top200
print(top_200.columns)
print('\n')
print(bottom_200.columns)

Index(['id', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed',
       'property_type', 'room_type', 'latitude', 'longitude', 'price',
       'minimum_nights', 'maximum_nights', 'accommodates', 'bedrooms', 'beds',
       'number_of_reviews', 'number_of_reviews_ltm', 'reviews_per_month',
       'review_scores_rating', 'review_scores_cleanliness',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'host_is_superhost', 'host_response_rate',
       'host_listings_count', 'availability_365', 'price_normalized',
       'weighted_score'],
      dtype='object')


Index(['id', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed',
       'property_type', 'room_type', 'latitude', 'longitude', 'price',
       'minimum_nights', 'maximum_nights', 'accommodates', 'bedrooms', 'beds',
       'number_of_reviews', 'number_of_reviews_ltm', 'reviews_per_month',
       'review_scores_rating', 'review_scores_cleanliness',
       'review_scores_communicati

In [97]:
top_200.head()

,id,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,latitude,longitude,price,minimum_nights,maximum_nights,accommodates,bedrooms,beds,number_of_reviews,number_of_reviews_ltm,reviews_per_month,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_value,host_is_superhost,host_response_rate,host_listings_count,availability_365,price_normalized,weighted_score
16063,968720450586425984.00,Sant Gervasi - Galvany,Sarrià-Sant Gervasi,Entire rental unit,Entire home/apt,41.39,2.14,248.00,76.00,330.00,6.00,3.00,4.00,0.00,0.00,1.86,4.49,4.62,4.51,4.68,4.26,f,97%,97.00,328.00,0.28,29.97
9444,50861575.00,la Sagrada Família,Eixample,Entire serviced apartment,Entire home/apt,41.40,2.18,122.00,76.00,1124.00,6.00,3.00,3.00,2.00,2.00,0.29,4.50,4.00,5.00,4.50,5.00,f,98%,97.00,76.00,0.66,29.88
9603,51759096.00,Sant Gervasi - Galvany,Sarrià-Sant Gervasi,Entire rental unit,Entire home/apt,41.40,2.15,148.00,76.00,330.00,6.00,3.00,3.00,0.00,0.00,1.00,4.25,4.33,4.50,4.86,4.20,f,97%,97.00,106.00,0.58,29.77
12395,744806258021033088.00,la Vila de Gràcia,Gràcia,Entire rental unit,Entire home/apt,41.40,2.16,208.00,76.00,330.00,4.00,3.00,3.00,0.00,0.00,1.61,4.77,4.84,4.71,4.93,4.58,f,97%,97.00,289.00,0.40,29.41
9915,52933728.00,la Vila de Gràcia,Gràcia,Entire rental unit,Entire home/apt,41.40,2.16,159.27,76.00,330.00,2.00,1.00,2.82,0.00,0.00,1.09,4.64,4.64,4.73,4.78,4.51,f,97%,97.00,43.00,0.55,28.78


In [98]:
bottom_200.head()

,id,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,latitude,longitude,price,minimum_nights,maximum_nights,accommodates,bedrooms,beds,number_of_reviews,number_of_reviews_ltm,reviews_per_month,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_value,host_is_superhost,host_response_rate,host_listings_count,availability_365,price_normalized,weighted_score
16735,1016449343214723584.00,el Baix Guinardó,Horta-Guinardó,Private room in rental unit,Private room,41.41,2.16,55.00,1.00,365.00,1.00,1.00,0.00,2.00,2.00,1.28,4.50,5.00,4.50,5.00,4.50,f,100%,1.00,261.00,0.86,1.32
14642,900907375525248000.00,la Nova Esquerra de l'Eixample,Eixample,Private room in rental unit,Private room,41.39,2.14,60.00,1.00,25.00,1.00,1.00,0.00,20.00,20.00,2.14,4.40,4.40,4.60,4.65,4.55,f,100%,1.00,55.00,0.85,1.33
5844,31884524.00,el Poblenou,Sant Martí,Private room in rental unit,Private room,41.40,2.20,45.00,1.00,365.00,1.00,1.00,0.00,10.00,10.00,4.50,4.60,4.60,4.90,4.70,4.50,f,100%,1.00,238.00,0.89,1.34
14457,893890275442034048.00,el Raval,Ciutat Vella,Private room in rental unit,Private room,41.38,2.17,70.00,1.00,1125.00,1.00,1.00,0.00,2.00,2.00,0.24,5.00,5.00,5.00,5.00,4.50,f,38%,1.00,349.00,0.82,1.34
12710,763883578421275520.00,Pedralbes,Les Corts,Private room in rental unit,Private room,41.39,2.13,65.00,1.00,30.00,1.00,1.00,0.00,32.00,32.00,2.94,4.79,4.91,4.91,4.84,4.69,f,100%,1.00,0.00,0.83,1.36


In [99]:
# COMPARACION DEL TIPO DE PROPIEDADES DE ALQUILER

property_type_top200 = top_200.groupby('property_type')['property_type'].value_counts()

property_type_bottom200 = bottom_200.groupby('property_type')['property_type'].value_counts()


# Creamos las figuras
fig1= px.bar(x=property_type_top200.index, y=property_type_top200.values, labels={'x': 'Tipo de Propiedad', 'y': 'Número'}, title= 'Número de tipos de propiedades- TOP_200', template='plotly')

fig2= px.bar(x=property_type_bottom200.index,y=property_type_bottom200.values, labels={'x': 'Tipo de Propiedad', 'y': 'Número'}, title= 'Número de tipos de propiedades- BOTTOM_200')

# Crear subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=('Top 200', 'Bottom 200'))

# Añadir las trazas del primer gráfico de barras a la primera columna del subplot
for trace in fig1.data:
    fig.add_trace(trace, row=1, col=1)

# Añadir las trazas del segundo gráfico de barras a la segunda columna del subplot
for trace in fig2.data:
    fig.add_trace(trace, row=1, col=2)

# Actualizar el layout del gráfico
fig.update_layout(title_text='Número de tipos de propiedades: Top 200 vs Bottom 200')

# Mostrar el gráfico
fig.show()

In [100]:
# COMPARACION DEL TIPO DE HABITACION

room_type_top200 = top_200.groupby('room_type')['room_type'].value_counts()

room_type_bottom200 = bottom_200.groupby('room_type')['room_type'].value_counts()


# Creamos las figuras
fig1= px.bar(x=room_type_top200.index, y=room_type_top200.values, labels={'x': 'Tipo de Habitación', 'y': 'Número'}, title= 'Número de tipos de habitaciones - TOP_200', template='plotly')

fig2= px.bar(x=room_type_bottom200.index,y=room_type_bottom200.values, labels={'x': 'Tipo de Habitación', 'y': 'Número'}, title= 'Número de tipos de habitaciones - BOTTOM_200', template='plotly')

# Crear subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=('Top 200', 'Bottom 200'))

# Añadir las trazas del primer gráfico de barras a la primera columna del subplot
for trace in fig1.data:
    fig.add_trace(trace, row=1, col=1)

# Añadir las trazas del segundo gráfico de barras a la segunda columna del subplot
for trace in fig2.data:
    fig.add_trace(trace, row=1, col=2)

# Actualizar el layout del gráfico
fig.update_layout(title_text='Número de tipos de habitaciones: Top 200 vs Bottom 200')

# Mostrar el gráfico
fig.show()

In [101]:
# PRECIO PROMEDIO POR TIPO DE TO

In [102]:
# COMPARACION DE NOCHES MINIMAS POR TIPO DE PROPIEDAD

min_nights_property_top200 = top_200.groupby('property_type')['minimum_nights'].mean()
min_nights_property_bottom200 = bottom_200.groupby('property_type')['minimum_nights'].mean()

# Creamos las figuras
fig1= px.bar(x=min_nights_property_top200.index, y=min_nights_property_top200.values, labels={'x': 'Tipo de Propiedad', 'y': 'Mínimo de Noches Medio'}, title= 'Minimo de noches medio por tipo de propiedad - TOP_200')

fig2= px.bar(x=min_nights_property_bottom200.index, y=min_nights_property_bottom200.values, labels={'x': 'Tipo de Propiedad', 'y': 'Mínimo de Noches Medio'}, title= 'Minimo de noches medio por tipo de propiedad - BOTTOM_200')

# Crear subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=('Top 200', 'Bottom 200'))

# Añadir las trazas del primer gráfico de barras a la primera columna del subplot
for trace in fig1.data:
    fig.add_trace(trace, row=1, col=1)

# Añadir las trazas del segundo gráfico de barras a la segunda columna del subplot
for trace in fig2.data:
    fig.add_trace(trace, row=1, col=2)

# Actualizar el layout del gráfico
fig.update_layout(title_text='Comparación de promedio de noches mínimas: Top 200 vs Bottom 200')

# Mostrar el gráfico
fig.show()

In [103]:
# COMPARACION DE PRECIOS POR TIPO DE PROPIEDAD

# Crear el primer gráfico de barras con Plotly Express
fig1 = px.histogram (top_200, x = 'price', color='property_type', title='Comparación de Precios por Tipo de Propiedad')

# Crear el segundo gráfico de barras con Plotly Express
fig2 = px.histogram(bottom_200, x = 'price', color='property_type',title='Comparación de Precios por Tipo de Propiedad')

# Crear subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=('Top 200', 'Bottom 200'))

# Añadir las trazas del primer gráfico de barras a la primera columna del subplot
for trace in fig1.data:
    fig.add_trace(trace, row=1, col=1)

# Añadir las trazas del segundo gráfico de barras a la segunda columna del subplot
for trace in fig2.data:
    fig.add_trace(trace, row=1, col=2)

# Actualizar el layout del gráfico
fig.update_layout(title_text='Comparación de Precios por Tipo de Propiedad: Top 200 vs Bottom 200')

# Mostrar el gráfico
fig.show()

In [104]:
# COMPARACION DE PUNTUAJES DE RESEÑAR POR BARRIO

# Creamos el primer grafico
fig1 = px.bar(top_200, x='neighbourhood_group_cleansed', y='review_scores_cleanliness',
                  title='Puntajes de Limpieza por Barrio', color='neighbourhood_group_cleansed')

# Creamos el segundo grafico
fig2 = px.bar(bottom_200, x='neighbourhood_group_cleansed', y='review_scores_cleanliness',
                  title='Puntajes de Limpieza por Barrio',color='neighbourhood_group_cleansed')

# Crear subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=('Top 200', 'Bottom 200'))

# Añadir las trazas del primer gráfico de barras a la primera columna del subplot
for trace in fig1.data:
    fig.add_trace(trace, row=1, col=1)

# Añadir las trazas del segundo gráfico de barras a la segunda columna del subplot
for trace in fig2.data:
    fig.add_trace(trace, row=1, col=2)

# Actualizar el layout del gráfico
fig.update_layout(title_text='Comparación de Puntuaje de Limpieza por Barrio: Top 200 vs Bottom 200')

# Mostrar el gráfico
fig.show()

In [105]:
# COMPARACION DE PUNTUAJES DE RESEÑAR POR BARRIO

# Creamos el primer grafico
fig1 = px.histogram(top_200, x='neighbourhood_group_cleansed', y='review_scores_cleanliness',
                  title='Puntajes de Limpieza por Barrio', color='neighbourhood_group_cleansed')

# Creamos el segundo grafico
fig2 = px.histogram(bottom_200, x='neighbourhood_group_cleansed', y='review_scores_cleanliness',
                  title='Puntajes de Limpieza por Barrio',color='neighbourhood_group_cleansed')

# Crear subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=('Top 200', 'Bottom 200'))

# Añadir las trazas del primer gráfico de barras a la primera columna del subplot
for trace in fig1.data:
    fig.add_trace(trace, row=1, col=1)

# Añadir las trazas del segundo gráfico de barras a la segunda columna del subplot
for trace in fig2.data:
    fig.add_trace(trace, row=1, col=2)

# Actualizar el layout del gráfico
fig.update_layout(title_text='Comparación de Puntuaje de Limpieza por Barrio: Top 200 vs Bottom 200')

# Mostrar el gráfico
fig.show()

In [106]:
# Calcular la frecuencia para los 50 peores listings
feq_bottom = bottom_200['neighbourhood_group_cleansed'].value_counts().sort_values(ascending=True)

# Calcular la frecuencia para los 50 mejores listings
feq_top = top_200['neighbourhood_group_cleansed'].value_counts().sort_values(ascending=True)

# Crear subplots con 1 fila y 2 columnas
fig = make_subplots(rows=1, cols=2, subplot_titles=("Bottom 200 Listings", "Top 200 Listings"))

# Gráfico para los 200 peores listings
fig.add_trace(go.Bar(
    y=feq_bottom.index,
    x=feq_bottom.values,
    orientation='h',
    marker=dict(color='blue'),
    name='Bottom 50'
), row=1, col=1)

# Gráfico para los 200 mejores listings
fig.add_trace(go.Bar(
    y=feq_top.index,
    x=feq_top.values,
    orientation='h',
    marker=dict(color='blue'),
    name='Top 50'
), row=1, col=2)

# Actualizar el layout
fig.update_layout(
    title_text="Lista del número de grupos vecinales",
    height=600,
    width=1200,
    showlegend=False
)

# Mostrar el gráfico
fig.show()

In [107]:
top_200.head()

,id,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,latitude,longitude,price,minimum_nights,maximum_nights,accommodates,bedrooms,beds,number_of_reviews,number_of_reviews_ltm,reviews_per_month,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_value,host_is_superhost,host_response_rate,host_listings_count,availability_365,price_normalized,weighted_score
16063,968720450586425984.00,Sant Gervasi - Galvany,Sarrià-Sant Gervasi,Entire rental unit,Entire home/apt,41.39,2.14,248.00,76.00,330.00,6.00,3.00,4.00,0.00,0.00,1.86,4.49,4.62,4.51,4.68,4.26,f,97%,97.00,328.00,0.28,29.97
9444,50861575.00,la Sagrada Família,Eixample,Entire serviced apartment,Entire home/apt,41.40,2.18,122.00,76.00,1124.00,6.00,3.00,3.00,2.00,2.00,0.29,4.50,4.00,5.00,4.50,5.00,f,98%,97.00,76.00,0.66,29.88
9603,51759096.00,Sant Gervasi - Galvany,Sarrià-Sant Gervasi,Entire rental unit,Entire home/apt,41.40,2.15,148.00,76.00,330.00,6.00,3.00,3.00,0.00,0.00,1.00,4.25,4.33,4.50,4.86,4.20,f,97%,97.00,106.00,0.58,29.77
12395,744806258021033088.00,la Vila de Gràcia,Gràcia,Entire rental unit,Entire home/apt,41.40,2.16,208.00,76.00,330.00,4.00,3.00,3.00,0.00,0.00,1.61,4.77,4.84,4.71,4.93,4.58,f,97%,97.00,289.00,0.40,29.41
9915,52933728.00,la Vila de Gràcia,Gràcia,Entire rental unit,Entire home/apt,41.40,2.16,159.27,76.00,330.00,2.00,1.00,2.82,0.00,0.00,1.09,4.64,4.64,4.73,4.78,4.51,f,97%,97.00,43.00,0.55,28.78


In [108]:
# GRAFICAMOS CON TOP200 - MEJORES

import folium
from folium.plugins import FastMarkerCluster

# Realizamos un mapa de las zonas que tenemos de acuerdo a cada top.
lats2018 = top_200['latitude'].tolist()
lons2018 = top_200['longitude'].tolist()
locations = list(zip(lats2018, lons2018))

map1 = folium.Map(location=[41.3851, 2.1734], zoom_start=11.5)
FastMarkerCluster(data=locations).add_to(map1)
map1

In [109]:
# GRAFICAMOS CON TOP200 - PEORES

import folium
from folium.plugins import FastMarkerCluster

# Realizamos un mapa de las zonas que tenemos de acuerdo a cada top.
lats2018 = bottom_200['latitude'].tolist()
lons2018 = bottom_200['longitude'].tolist()
locations = list(zip(lats2018, lons2018))

map1 = folium.Map(location=[41.3851, 2.1734], zoom_start=11.5)
FastMarkerCluster(data=locations).add_to(map1)
map1

In [110]:
# MEJORES Y PEORES EN UN SOLO MAPA

import folium

# Crear un mapa para las ubicaciones del top 200
map_combined = folium.Map(location=[41.3851, 2.1734], zoom_start=11.5)

# Agregar marcadores rojos para las ubicaciones del top 50 peores
for index, row in bottom_200.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], icon=folium.Icon(color='red')).add_to(map_combined)

# Agregar marcadores verdes para las ubicaciones del top 50 mejores
for index, row in top_200.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], icon=folium.Icon(color='green')).add_to(map_combined)

# Mostrar el mapa combinado
from IPython.display import display
display(map_combined)


In [111]:
# guardamos los dataframes en nuestra carpetas data
import os

# Definiimos el lugar donde vamos a guardar cada uno de los dataframes
base_path = '/Users/juliobrionesmorales/Documents/GitHub/Airbnb_Barcelona/data/4. top200'
folder_path = os.path.join(base_path)

# Guarda cada DataFrame como un archivo CSV en la carpeta específica
top_200.to_csv(os.path.join(folder_path, 'top_200.csv'), index=False)
bottom_200.to_csv(os.path.join(folder_path, 'bottom_200.csv'), index=False)


# **CONTRASTE DE HIPOSTESIS**

Implementación del A/B Testing
Para cada una de estas preguntas, el proceso de A/B Testing implicaría:

1. Definir las variables de interés: Determinar cuáles variables actuarán como independientes (grupos A y B) y cuáles como dependientes (resultado del test).
2. Recolección de datos adecuada: Asegurarse de que los datos para cada grupo estén bien definidos y sean comparables.
3. Comprobación de supuestos: Verificar la normalidad de las distribuciones y la homogeneidad de las varianzas para decidir el tipo de prueba estadística a aplicar.
4. Realización de pruebas estadísticas: Ejecutar la prueba seleccionada para comparar los grupos.
5. Interpretación de resultados: Determinar si las diferencias son estadísticamente significativas y qué implican para el negocio.

## **CONTRASTE 1**

Vamos hacer este contraste de hipotesis con los dataframes top200, bottom_200 y df_listings.

### CONTRASTE 1 - TOP200
1. **Hipotesis nula (Ho)**: el tipo de propiedad no tiene un impacto significativo en el precio de la vivienda.

2. **Hipotesis alternativa (H1)**: el tipo de propiedad tiene un impacto significativo en el precio de la vivienda.

In [112]:
# Vemos la asimetria de los datos con 'skew' (vemos si siguen una distribución normal)
skew(top_200['price'])

1.0923659486982387

Como la asimetria es positiva, hay mas peso en la cola de la derecha de la distribución.

In [113]:
# Creamos un dataframe con los datos a analizar
top_200_df = top_200[['property_type', 'price']].dropna()

In [114]:
top_200_df['property_type'].value_counts()

property_type
Entire rental unit             180
Entire serviced apartment       15
Private room in rental unit      3
Entire condo                     2
Name: count, dtype: int64

In [115]:
# Agrupamos por tipo de propiedad y extraemos el precio convertiendolo en un array
groups = [group['price'].values for name, group in top_200_df.groupby('property_type')]
groups

[array([100., 100.]),
 array([248.        , 148.        , 208.        , 159.27272727,
         50.        ,  50.        ,  50.        ,  78.        ,
        184.81818182,  81.        , 125.        , 110.        ,
        110.        , 125.        , 182.        , 110.        ,
        224.        , 341.38636364, 110.        , 129.        ,
        120.        , 205.        , 188.        , 120.        ,
        218.        , 134.        , 135.        , 183.        ,
        229.        , 271.09090909, 132.        , 132.        ,
        182.        , 132.        , 188.        , 132.        ,
        132.        , 132.        ,  92.        , 157.        ,
        260.        , 260.        , 108.        , 158.        ,
        120.        , 120.        , 125.        , 260.        ,
        219.        , 185.81818182, 260.        , 341.38636364,
        341.38636364, 100.        , 103.        , 130.        ,
        123.        , 150.        , 156.        , 161.        ,
        106.      

Realizamos un ANOVA de una vía para comparar las medias delos valores de 'price' para cada uno de tipo de propiedad almacenados en nuestro lista de groups. Con el resultado de f_stat y p_value podemos determinar si existe una diferencia significativa en los precios entrelos diferentes tipos de propiedades. 

In [116]:
# Realizamos ANOVA
f_stat, p_value = stats.f_oneway(*groups)
print(p_value)

0.0007191067328879108


Dado que el p-valor es menor de que el nivel de significación (0.05) aceptamos la hipotesis alterna (H1).

- H1: el tipo de propiedad tiene un impacto significativo en el precio de la vivienda.

### CONTRASTE 1 - BOTTOM_200
1. **Hipotesis nula (Ho)**: el tipo de propiedad no tiene un impacto significativo en el precio de la vivienda.

2. **Hipotesis alternativa (H1)**: el tipo de propiedad tiene un impacto significativo en el precio de la vivienda.

In [117]:
# Vemos la asimetria de los datos con 'skew' (vemos si siguen una distribución normal)
skew(bottom_200['price'])

2.0492333724639713

Como la asimetria es positiva, hay mas peso en la cola de la derecha de la distribución.

In [118]:
# Creamos un dataframe con los datos a analizar
bottom_200_df = bottom_200[['property_type', 'price']].dropna()

In [119]:
bottom_200_df['property_type'].value_counts()

property_type
Private room in rental unit          159
Private room in home                   7
Entire rental unit                     7
Private room in condo                  5
Room in hotel                          5
Private room in casa particular        4
Private room in bed and breakfast      3
Shared room in hostel                  2
Room in boutique hotel                 2
Entire loft                            2
Private room in guesthouse             1
Shared room in rental unit             1
Private room                           1
Shared room in bed and breakfast       1
Name: count, dtype: int64

In [120]:
# Agrupamos por tipo de propiedad y extraemos el precio convertiendolo en un array
groups = [group['price'].values for name, group in bottom_200_df.groupby('property_type')]
groups

[array([ 91., 110.]),
 array([120.,  82., 210.,  82.,  73., 114.,  68.]),
 array([40.]),
 array([46., 40., 40.]),
 array([70., 34., 53., 95.]),
 array([39., 68., 32., 50., 81.]),
 array([35.]),
 array([48., 67., 51., 66., 48., 67., 60.]),
 array([ 55.        ,  60.        ,  45.        ,  70.        ,
         65.        ,  70.        ,  42.        ,  35.        ,
         30.        ,  43.        ,  40.        ,  71.        ,
         52.        ,  53.        , 200.        ,  67.        ,
         50.        ,  68.        , 130.        , 150.        ,
        100.        , 100.        , 100.        ,  30.        ,
         56.        ,  55.        ,  50.        ,  49.        ,
         70.        ,  40.        ,  50.        ,  66.        ,
         50.        , 215.09090909,  30.        ,  92.        ,
         79.        ,  80.        , 150.        ,  28.        ,
        120.        ,  44.        ,  64.        ,  36.        ,
         43.        , 197.        ,  40.        , 120.   

Realizamos un ANOVA de una vía para comparar las medias delos valores de 'price' para cada uno de tipo de propiedad almacenados en nuestro lista de groups. Con el resultado de f_stat y p_value podemos determinar si existe una diferencia significativa en los precios entrelos diferentes tipos de propiedades. 

In [121]:
# Realizamos ANOVA
f_stat, p_value = stats.f_oneway(*groups)
print(p_value)

0.0017804779917325435


Dado que el p-valor es menor de que el nivel de significación (0.05) aceptamos la hipotesis alterna (H1).

- H1: el tipo de propiedad tiene un impacto significativo en el precio de la vivienda.

### CONTRASTE 1 - DF_LISTINGS
1. **Hipotesis nula (Ho)**: el tipo de propiedad no tiene un impacto significativo en el precio de la vivienda.

2. **Hipotesis alternativa (H1)**: el tipo de propiedad tiene un impacto significativo en el precio de la vivienda.

In [122]:
# Vemos la asimetria de los datos con 'skew' (vemos si siguen una distribución normal)
skew(df_listings['price'])

0.8369389713897533

Como la asimetria es positiva, hay mas peso en la cola de la derecha de la distribución.

In [123]:
# Creamos un dataframe con los datos a analizar
df_listings_df = df_listings[['property_type', 'price']].dropna()

In [124]:
df_listings_df['property_type'].value_counts()

property_type
Entire rental unit                    9681
Private room in rental unit           5674
Entire serviced apartment              465
Room in hotel                          402
Entire condo                           395
Entire loft                            289
Private room in condo                  213
Private room in hostel                 199
Room in boutique hotel                 193
Private room in home                   187
Private room in bed and breakfast       91
Private room in casa particular         80
Shared room in hostel                   80
Entire home                             78
Shared room in rental unit              41
Private room in guest suite             39
Private room in serviced apartment      39
Private room in loft                    35
Room in hostel                          30
Entire vacation home                    30
Private room                            24
Entire villa                            23
Entire guest suite                      

In [125]:
# Agrupamos por tipo de propiedad y extraemos el precio convertiendolo en un array
groups = [group['price'].values for name, group in df_listings_df.groupby('property_type')]
groups

[array([237.]),
 array([341.38636364, 200.        , 134.        , 341.38636364,
         84.        , 341.38636364, 196.18181818, 149.        ,
        341.38636364,  99.90909091, 218.        ,  95.        ,
        127.        , 129.        ]),
 array([101.        ,  71.        , 150.        ,  62.        ,
        101.        ,  91.27272727, 145.81818182, 161.        ,
        108.        ]),
 array([ 80.        ,  99.        ,  99.        ,  85.        ,
        341.38636364, 112.        , 103.        ]),
 array([41.]),
 array([180.72727273]),
 array([ 95.54545455,  79.09090909, 275.        ]),
 array([186.        , 113.        , 145.        , 309.        ,
        177.        , 125.        , 341.38636364,  42.        ,
         92.        , 300.        , 300.        , 170.        ,
        196.        , 175.        , 109.        ,  95.        ,
        156.        , 212.        , 129.        , 275.        ,
        130.        , 180.        , 196.        , 285.        ,
        144

Realizamos un ANOVA de una vía para comparar las medias delos valores de 'price' para cada uno de tipo de propiedad almacenados en nuestro lista de groups. Con el resultado de f_stat y p_value podemos determinar si existe una diferencia significativa en los precios entrelos diferentes tipos de propiedades. 

In [126]:
# Realizamos ANOVA
f_stat, p_value = stats.f_oneway(*groups)
print(p_value)

0.0


Dado que el p-valor es menor de que el nivel de significación (0.05) rechazamos la hipotesis nula (H0), y por lo que aceptamos la hipotesis alternativa (H1).

- H1: el tipo de propiedad tiene un impacto significativo en el precio de la vivienda.

# CONTRASTE 2

### CONTRASTE 2 - TOP_200

1. **Hipotesis nula (Ho)**: no hay diferencia significativa en la aceptación del alquiler entre propiedades con diferentes capacidades de acomodación. 

2. **Hipotesis alternativa (H1)**: existen una diferencia significativa en la aceptación del alquiler entre propiedades con diferentes capacidades de acomodación

In [127]:
# Comprobamos la asimetria de los datos con 'skew'
print('Asimetria de acommodates es:')
print(skew(top_200['accommodates']))
print('\n Asimetria de availability_365:')
print(skew(top_200['availability_365']))

Asimetria de acommodates es:
-1.1969846767600567

 Asimetria de availability_365:
-0.4217486256345294


Como la asimetria es negativa, hay más peso en la cola de la izquierda de la distribución, pero es cierto que estan muy cerca a una distribución normal. Tras analizar el caso, hemos decidido trazar el camino de analisis no paramétricos.

Como nuestra hipotesis es de diferencia, y no son muestras pareadas, elegimos el test Mann Whitney para poder comprobar nuestra hipótesis.

In [128]:
# MANN WHITNEY


from scipy.stats import mannwhitneyu
stat , p_valor = mannwhitneyu(top_200['accommodates'], top_200['availability_365'])

print(f"Coeficiente Mann Whitney: {stat}")
print(f"p-value: {p_valor}")

# Mostramos resultado
print(f"El p-valor obtenido en el test de Mann Whitney es de {p_valor}")

Coeficiente Mann Whitney: 2918.0
p-value: 4.477555774696787e-50
El p-valor obtenido en el test de Shapiro-Wilk es de 4.477555774696787e-50


##### Como el p-valor obtenido es menor que el nivel de significación (0.05), rechazamos la hipotesis nula y aceptamos que existen una diferencia significativa en la aceptación del alquiler entre propiedades con diferentes capacidades de acomodación

### CONTRASTE 2 - BOTTOM_200
1. **Hipotesis nula (Ho)**: no hay diferencia significativa en la aceptación del alquiler entre propiedades con diferentes capacidades de acomodación. 

2. **Hipotesis alternativa (H1)**: existen una diferencia significativa en la aceptación del alquiler entre propiedades con diferentes capacidades de acomodación

In [129]:
# Comprobamos la asimetria de los datos con 'skew'
print('Asimetria de acommodates es:')
print(skew(bottom_200['accommodates']))
print('\n Asimetria de availability_365:')
print(skew(bottom_200['availability_365']))

Asimetria de acommodates es:
1.017548873515413

 Asimetria de availability_365:
0.2189434220431585


Dada que acommodates muestra una asimetrias signiticativamente sesgada, vamos a realizar test no paramétricos para evaluar nuestra hipotesis nula.


Como nuestra hipotesis es de diferencia, y no son muestras pareadas, elegimos el test Mann Whitney para poder comprobar nuestra hipótesis.

In [130]:
# MANN WHITNEY


from scipy.stats import mannwhitneyu
stat , p_valor = mannwhitneyu(bottom_200['accommodates'], bottom_200['availability_365'])

print(f"Coeficiente Mann Whitney: {stat}")
print(f"p-value: {p_valor}")

# Mostramos resultado
print(f"El p-valor obtenido en el test de Mann Whitney es de {p_valor}")

Coeficiente Mann Whitney: 4990.0
p-value: 2.078055775333708e-40
El p-valor obtenido en el test de Shapiro-Wilk es de 2.078055775333708e-40


##### Como el p-valor obtenido es menor que el nivel de significación (0.05), rechazamos la hipotesis nula y aceptamos que existen una diferencia significativa en la aceptación del alquiler entre propiedades con diferentes capacidades de acomodación

### CONTRASTE 2 - DF_LISTINGS
1. **Hipotesis nula (Ho)**: no hay diferencia significativa en la aceptación del alquiler entre propiedades con diferentes capacidades de acomodación. 

2. **Hipotesis alternativa (H1)**: existen una diferencia significativa en la aceptación del alquiler entre propiedades con diferentes capacidades de acomodación

In [131]:
# Comprobamos la asimetria de los datos con 'skew'
print('Asimetria de acommodates es:')
print(skew(df_listings['accommodates']))
print('\n Asimetria de availability_365:')
print(skew(df_listings['availability_365']))

Asimetria de acommodates es:
0.5759355516920959

 Asimetria de availability_365:
-0.05738325463519655


Dado que la asimetria se esta dentro los umbrales permitidos, vamos a realizar tests paramétricos para poder evaluar nuestra hipótesis nula.


Como nuestra hipotesis nula es de diferencia, realizamos un test de anova. 

In [132]:
from scipy.stats import f_oneway

# Calcular el coeficiente de correlación de Pearson
corr, p_value = f_oneway(df_listings['accommodates'], df_listings['availability_365'])

# Mostramos los resultados
print(f'El p-valor es de: {p_value}')

# Decisión
alpha = 0.05
if p_value < alpha:
    print("Rechazamos la hipótesis nula (H0). Existe una correlación significativa entre la capacidad de alojamiento y la disponibilidad del alquiler.")
else:
    print("No podemos rechazar la hipótesis nula (H0). No hay suficiente evidencia para decir que existe una correlación significativa entre la capacidad de alojamiento y la disponibilidad del alquiler.")

El p-valor es de: 0.0
Rechazamos la hipótesis nula (H0). Existe una correlación significativa entre la capacidad de alojamiento y la disponibilidad del alquiler.


##### Como el p-valor obtenido es menor que el nivel de significación (0.05), rechazamos la hipotesis nula y aceptamos que existen una diferencia significativa en la aceptación del alquiler entre propiedades con diferentes capacidades de acomodación

## **CONTRASTE 3**
1. **Hipotesis nula (Ho)**: no hay diferencia significativa en la aceptación del alquiler y el número mínimo de noches.  

2. **Hipotesis alternativa (H1)**: existe una diferencia significativa en la aceptación del alquiler y el número mínimo de noches. 


### CONTRASTE 3 - TOP_200

Para poder comprobar nuestra hipótesis vamos a tener en cuenta el minimum_nights y availability_365.

In [133]:
# Calculamos la asimetria de cada una de las columnas a analizar. 
print('Asimetria de minimum_nights es de:')
print(skew(top_200['minimum_nights']))
print('\n')
print('Asimetria de availability_365 es de:')
print(skew(top_200['availability_365']))

Asimetria de minimum_nights es de:
4.032320313868242


Asimetria de availability_365 es de:
-0.4217486256345294


Una de la columnas se encuentra dentro de los rango de aceptación de normalidad (availability_365) y la hora tiene una desproporcion hacia la derecha (minimum_nights) por lo que vamos a realizar un analisis no paramétrico.

In [134]:
# Prueba de Shapiro-Wilk para normalidad
stat, p_norm1 = stats.shapiro(top_200['minimum_nights'])
stat, p_norm2 = stats.shapiro(top_200['availability_365'])
print(f"Normalidad Grupo 1: p-value = {p_norm1}")
print(f"Normalidad Grupo 2: p-value = {p_norm2}")

Normalidad Grupo 1: p-value = 5.664787049232753e-27
Normalidad Grupo 2: p-value = 7.287925654964056e-06


Realizamos el test de shapiro para poder determinar la normalidad, y como el p-valor es menor que el nivel de significación (0.05) concluimos que tenemos que realizar un test no paramétrico.

In [135]:
# Realizamos el test mann whitney para poder comprobar nuestra hipótesis
from scipy.stats import mannwhitneyu

stats, p_value = mannwhitneyu(top_200['minimum_nights'],top_200['availability_365']) 

print(f'El p_valor es de: {p_value}')

El p_valor es de: 3.564377739679799e-47


##### Como el p-valor es menor que el nivel de significación, rechazamos la hipotesis nula (Ho) y aceptamos que existe una diferencia significativa en la aceptación del alquiler y el numero de mínimo de noches.

### CONTRASTE 3 - BOTTOM_200
Realizamos el mismo analisis anterior pero con el dataframe bottom_200.

In [136]:
# Calculamos la asimetria de cada una de las columnas a analizar. 
print('Asimetria de minimum_nights es de:')
print(skew(bottom_200['minimum_nights']))
print('\n')
print('Asimetria de availability_365 es de:')
print(skew(bottom_200['availability_365']))

Asimetria de minimum_nights es de:
1.78158623924552


Asimetria de availability_365 es de:
0.2189434220431585


Una de la columnas se encuentra dentro de los rango de aceptación de normalidad (availability_365) y la hora tiene una desproporcion hacia la derecha (minimum_nights) por lo que vamos a realizar un analisis no paramétrico.
Pero realizamos el test shapiro para comprobar la normalidad.

In [137]:
# Prueba de Shapiro-Wilk para normalidad
import scipy.stats as stats

stat, p_norm1 = stats.shapiro(bottom_200['minimum_nights'])
stat, p_norm2 = stats.shapiro(bottom_200['availability_365'])
print(f"Normalidad Grupo 1: p-value = {p_norm1}")
print(f"Normalidad Grupo 2: p-value = {p_norm2}")

Normalidad Grupo 1: p-value = 6.52858508370523e-22
Normalidad Grupo 2: p-value = 1.0032024516659632e-10


Como el p-valor es menor que el nivel de significación (0.05) nuestras variables no siguen una distribución normal, por lo que tendremos que realizar un tests no paramétricos. 

In [138]:
# Realizamos el test mann whitney para poder comprobar nuestra hipótesis
from scipy.stats import mannwhitneyu

stats, p_value = mannwhitneyu(bottom_200['minimum_nights'],bottom_200['availability_365']) 

print(f'El p_valor es de: {p_value}')

El p_valor es de: 1.2917257136019567e-40


##### Como el p-valor es menor que el nivel de significación, rechazamos la hipotesis nula (Ho) y aceptamos que existe una diferencia significativa en la aceptación del alquiler y el numero de mínimo de noches.

### CONTRASTE 3 - DF_LISTINGS
Realizamos el mismo analisis anterior pero con el dataframe bottom_200.

In [139]:
# Prueba de Shapiro-Wilk para normalidad
import scipy.stats as stats

stat, p_norm1 = stats.shapiro(df_listings['minimum_nights'])
stat, p_norm2 = stats.shapiro(df_listings['availability_365'])
print(f"Normalidad Grupo 1: p-value = {p_norm1}")
print(f"Normalidad Grupo 2: p-value = {p_norm2}")

Normalidad Grupo 1: p-value = 0.0
Normalidad Grupo 2: p-value = 0.0


/Users/juliobrionesmorales/Documents/1. FORMACION/3. DATA ANALYTICS /PROYECTOS_BOOTCAMP/PROYECTO FINAL 2/entorno/lib/python3.10/site-packages/scipy/stats/_morestats.py:1882: UserWarning:

p-value may not be accurate for N > 5000.



Realizamos test no paramétricos para poder comprobar nuestra hipótesis:

In [140]:
# Realizamos el test mann whitney para poder comprobar nuestra hipótesis
from scipy.stats import mannwhitneyu

stats, p_value = mannwhitneyu(df_listings['minimum_nights'],df_listings['availability_365']) 

print(f'El p_valor es de: {p_value}')

El p_valor es de: 0.0


##### Como el p-valor es menor que el nivel de significación, rechazamos la hipotesis nula (Ho) y aceptamos que existe una diferencia significativa en la aceptación del alquiler y el numero de mínimo de noches.

In [141]:
# De manera adicional podemos determinar si existe una correlación entre las noches minimas y availability_365 mediante test de spearman

# TOP_200


from scipy.stats import spearmanr

stats, p_value = spearmanr (top_200['minimum_nights'],top_200['availability_365']) 

print(f'El p_valor es de: {p_value}')
print(f'El stats es de: {stats}')

El p_valor es de: 1.6087094439499392e-06
El stats es de: -0.3316447756075365


In [142]:
# BOTTOM_200


from scipy.stats import spearmanr

stats, p_value = spearmanr (bottom_200['minimum_nights'],bottom_200['availability_365']) 

print(f'El p_valor es de: {p_value}')
print(f'El stats es de: {stats}')

El p_valor es de: 0.027426960270957968
El stats es de: -0.15596642899262597


In [143]:
# DF_LISTINGS


from scipy.stats import spearmanr

stats, p_value = spearmanr (df_listings['minimum_nights'],df_listings['availability_365']) 

print(f'El p_valor es de: {p_value}')
print(f'El stats es de: {stats}')

El p_valor es de: 1.1742749301697884e-67
El stats es de: 0.12720254009860332


## **CONTRASTE 4**
1. **Hipotesis nula (Ho)**: no existe una correlación entre el número de camas y el availability_365.

2. **Hipotesis alternativa (H1)**: existe una correlación entre el número de camas y el availability_365.

### CONTRASTE 4 - TOP_200

In [144]:
# Comprobamos la normalidad de nuestros datos con la Prueba de Shapiro-Wilk 
import scipy.stats as stats

stat, p_norm1 = stats.shapiro(top_200['beds'])
stat, p_norm2 = stats.shapiro(top_200['availability_365'])
print(f"Normalidad Grupo 1: p-value = {p_norm1}")
print(f"Normalidad Grupo 2: p-value = {p_norm2}")

Normalidad Grupo 1: p-value = 6.053252517290719e-13
Normalidad Grupo 2: p-value = 7.287925654964056e-06


In [145]:
# Como no siguen una distribución normal, realizamo un test de spearman para poder ver la correlación entre las varibles

from scipy.stats import spearmanr

stats, p_value = spearmanr (top_200['beds'],top_200['availability_365']) 

print(f'El p_valor es de: {p_value}')
print(f'El stats es de: {stats}')

El p_valor es de: 0.0032779240833870227
El stats es de: 0.20695886877715805


##### De acuerdo a los datos, rechazamos la hipotesis nula y aceptamos que existe una correlación entre el número de camas y el availability_365. El stats establece una correlación, pero una correlación debil.

### BOTTOM_200

In [146]:
# Comprobamos la normalidad de nuestros datos con la Prueba de Shapiro-Wilk 
import scipy.stats as stats

stat, p_norm1 = stats.shapiro(bottom_200['beds'])
stat, p_norm2 = stats.shapiro(bottom_200['availability_365'])
print(f"Normalidad Grupo 1: p-value = {p_norm1}")
print(f"Normalidad Grupo 2: p-value = {p_norm2}")

Normalidad Grupo 1: p-value = 7.704195435431243e-21
Normalidad Grupo 2: p-value = 1.0032024516659632e-10


In [147]:
# Como no siguen una distribución normal, realizamo un test de spearman para poder ver la correlación entre las varibles

from scipy.stats import spearmanr

stats, p_value = spearmanr (bottom_200['beds'],bottom_200['availability_365']) 

print(f'El p_valor es de: {p_value}')
print(f'El stats es de: {stats}')

El p_valor es de: 0.9133412254358098
El stats es de: -0.007743517140127181



##### Como el p-valor es mayor que 0.05, no rechazamos la hipotesis nula (no existe una correlación entre el número de camas y el availability_365). Esto significa que no hay suficiente evidencia para concluir que no hay correlación entre el número de camas y availability_365.

##### Como el coeficiente de correlación nos sale tan cercano al cero, indica que prácticamente no hay una correlación monotónica entre el número de camas y availability_365.

### DF_LISTINGS

In [148]:
# Comprobamos la normalidad de nuestros datos con la Prueba de Shapiro-Wilk 
import scipy.stats as stats

stat, p_norm1 = stats.shapiro(df_listings['beds'])
stat, p_norm2 = stats.shapiro(df_listings['availability_365'])
print(f"Normalidad Grupo 1: p-value = {p_norm1}")
print(f"Normalidad Grupo 2: p-value = {p_norm2}")

Normalidad Grupo 1: p-value = 0.0
Normalidad Grupo 2: p-value = 0.0


In [149]:
# Como no siguen una distribución normal, realizamo un test de spearman para poder ver la correlación entre las varibles

from scipy.stats import spearmanr

stats, p_value = spearmanr (df_listings['beds'],df_listings['availability_365']) 

print(f'El p_valor es de: {p_value}')
print(f'El stats es de: {stats}')

El p_valor es de: 1.5242685176988299e-33
El stats es de: -0.0885255167047574


##### De acuerdo a los datos, rechazamos la hipotesis nula y aceptamos que existe una correlación entre el número de camas y el availability_365. El stats establece una correlación, pero una correlación super debil.

## **CONTRASTE 5**
1. **Hipotesis nula (Ho)**: no existe una correlación entre el número de habitaciones y el precio de la vivienda.

2. **Hipotesis alternativa (H1)**: existe una correlación entre el número de habitaciones y el precio de la vivienda.

### CONTRASTE 5 - TOP_200

In [150]:
top_200.columns

Index(['id', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed',
       'property_type', 'room_type', 'latitude', 'longitude', 'price',
       'minimum_nights', 'maximum_nights', 'accommodates', 'bedrooms', 'beds',
       'number_of_reviews', 'number_of_reviews_ltm', 'reviews_per_month',
       'review_scores_rating', 'review_scores_cleanliness',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'host_is_superhost', 'host_response_rate',
       'host_listings_count', 'availability_365', 'price_normalized',
       'weighted_score'],
      dtype='object')

In [151]:
# Comprobamos la normalidad de nuestros datos
import scipy.stats as stats

stat, p_norm1 = stats.shapiro(top_200['bedrooms'])
stat, p_norm2 = stats.shapiro(top_200['price'])
print(f"Normalidad Grupo 1: p-value = {p_norm1}")
print(f"Normalidad Grupo 2: p-value = {p_norm2}")


Normalidad Grupo 1: p-value = 1.1319150811285568e-21
Normalidad Grupo 2: p-value = 9.22243259537936e-09


Dado que los p-valores son menores que el nivel de significación, realizamos tests no paramétricos para poder comprobar nuestra normalidad. 

In [152]:
# Como no siguen una distribución normal, realizamo un test de spearman para poder ver la correlación entre las varibles

from scipy.stats import spearmanr

stats, p_value = spearmanr (top_200['bedrooms'],top_200['price']) 

print(f'El p_valor es de: {p_value}')
print(f'El stats es de: {stats}')

El p_valor es de: 0.6875944904177846
El stats es de: 0.028607789352073962


##### Dado que el p-valor es mayor que el nivel de significación, no podemos rechazar la hipotesis nula, y de acuerdo a los datos podemos decir que no existe una correlación entre el número de habitaciones y el precio.

### CONTRASTE 5 - BOTTOM_200

In [153]:
# Comprobamos la normalidad de nuestros datos
import scipy.stats as stats

stat, p_norm1 = stats.shapiro(bottom_200['bedrooms'])
stat, p_norm2 = stats.shapiro(bottom_200['price'])
print(f"Normalidad Grupo 1: p-value = {p_norm1}")
print(f"Normalidad Grupo 2: p-value = {p_norm2}")

Normalidad Grupo 1: p-value = 3.340926266857368e-25
Normalidad Grupo 2: p-value = 8.080054277535256e-16


Dado que los p-valores son menores que el nivel de significación, realizamos tests no paramétricos para poder comprobar nuestra normalidad. 

In [154]:
# Como no siguen una distribución normal, realizamo un test de spearman para poder ver la correlación entre las varibles

from scipy.stats import spearmanr

stats, p_value = spearmanr (bottom_200['bedrooms'],bottom_200['price']) 

print(f'El p_valor es de: {p_value}')
print(f'El stats es de: {stats}')

El p_valor es de: 0.11708249004912573
El stats es de: -0.11116642884920827


##### Dado que el p-valor es mayor que el nivel de significación, no podemos rechazar la hipotesis nula, y de acuerdo a los datos, no existe una correlación entre el número de habitaciones y el precio. Por otro lado, de acuerdo al stats exite un debil correlación negativa entre ambas variables. 

### CONTRASTE 5 - DF_LISTINGS

In [156]:
# Comprobamos la normalidad de nuestros datos
import scipy.stats as stats

stat1, p_norm1 = stats.shapiro(df_listings['bedrooms'])
stat2, p_norm2 = stats.shapiro(df_listings['price'])
print(f"Normalidad Grupo 1: p-value = {p_norm1}")
print(f"Normalidad Grupo 2: p-value = {p_norm2}")
print(f'El stats es de: {stat1}')
print(f'El stats es de: {stat2}')

Normalidad Grupo 1: p-value = 0.0
Normalidad Grupo 2: p-value = 0.0
El stats es de: 0.8649062514305115
El stats es de: 0.9317843914031982


/Users/juliobrionesmorales/Documents/1. FORMACION/3. DATA ANALYTICS /PROYECTOS_BOOTCAMP/PROYECTO FINAL 2/entorno/lib/python3.10/site-packages/scipy/stats/_morestats.py:1882: UserWarning:

p-value may not be accurate for N > 5000.



In [157]:
# Utilizamos la asimetria para recomprobar la normalidad
print('Asimetria de bedrooms es:')
print(skew(df_listings['bedrooms']))
print('\n Asimetria de price:')
print(skew(df_listings['price']))

Asimetria de bedrooms es:
0.4701738673370464

 Asimetria de price:
0.8369389713897533


Como una de las asimetrias se espaca de rango optimo, no guiamos por los test no paramétricos. 

In [158]:
# Como no siguen una distribución normal, realizamo un test de spearman para poder ver la correlación entre las varibles

from scipy.stats import spearmanr

stats, p_value = spearmanr (df_listings['bedrooms'],df_listings['price']) 

print(f'El p_valor es de: {p_value}')
print(f'El stats es de: {stats}')

El p_valor es de: 0.0
El stats es de: 0.47792179548783587


##### Como el p-valor es menor que el nivel de significacion, rechazamos la hipotesis nula, y de acuerdo a los datos, podemos establecer que existe una correlación entre el número de habitaciones y el precio de la vivienda. Por otro lado, si vemos el stas, podemos decir que existe una correlación positiva moderada entre ambas variables. 

## **CONTRASTE 6**
1. **Hipotesis nula (Ho)**: no existe una diferencia significativa entre la aceptación del alquiler de acuerdo al precio

2. **Hipotesis alternativa (H1)**: existe una diferencia significativa entre la aceptación del alquiler de acuerdo al precio

### CONTRASTE 6 - TOP_200

In [160]:
# Comprobamos la normalidad de nuestros datos
import scipy.stats as stats

stat1, p_norm1 = stats.shapiro(top_200['availability_365'])
stat2, p_norm2 = stats.shapiro(top_200['price'])
print(f"Normalidad Grupo 1: p-value = {p_norm1}")
print(f"Normalidad Grupo 2: p-value = {p_norm2}")

Normalidad Grupo 1: p-value = 7.287925654964056e-06
Normalidad Grupo 2: p-value = 9.22243259537936e-09


Como los p-valores son menores que el nivel de significación, de acuerdo a los datos, no siguen una distribución normal.

In [162]:
# MANN WHITNEY


from scipy.stats import mannwhitneyu
stat , p_valor = mannwhitneyu(top_200['availability_365'], top_200['price'])

# Mostramos resultado
print(f"El p-valor obtenido en el test de Mann Whitney de {p_valor}")

El p-valor obtenido en el test de Mann Whitney de 1.2975892639872694e-12


##### De acuerdo al p-valor del test, menor que el nivel de significaición, rechazamos la hipotesis nula, y existe una diferencia significativa entre la aceptación del alquiler de acuerdo al precio.

In [165]:
from scipy.stats import spearmanr

stats, p_value = spearmanr (top_200['availability_365'],top_200['price']) 

print(f'El p_valor es de: {p_value}')
print(f'El stats es de: {stats}')

El p_valor es de: 0.8323417067759342
El stats es de: -0.015062960531316729


### CONTRASTE 6 - BOTTOM_200

In [163]:
# Comprobamos la normalidad de nuestros datos
import scipy.stats as stats

stat1, p_norm1 = stats.shapiro(bottom_200['availability_365'])
stat2, p_norm2 = stats.shapiro(bottom_200['price'])
print(f"Normalidad Grupo 1: p-value = {p_norm1}")
print(f"Normalidad Grupo 2: p-value = {p_norm2}")

Normalidad Grupo 1: p-value = 1.0032024516659632e-10
Normalidad Grupo 2: p-value = 8.080054277535256e-16


Como los p-valores son menores que el nivel de significación, de acuerdo a los datos, no siguen una distribución normal.

In [164]:
# MANN WHITNEY


from scipy.stats import mannwhitneyu
stat , p_valor = mannwhitneyu(bottom_200['availability_365'], bottom_200['price'])

# Mostramos resultado
print(f"El p-valor obtenido en el test de Mann Whitney de {p_valor}")

El p-valor obtenido en el test de Mann Whitney de 1.0062217154707354e-07


##### De acuerdo al p-valor del test, menor que el nivel de significaición, rechazamos la hipotesis nula, y existe una diferencia significativa entre la aceptación del alquiler de acuerdo al precio

### CONTRASTE 6 - DF_LISTINGS_200

In [167]:
# Utilizamos la asimetria para recomprobar la normalidad
print('Asimetria de availability_365 es:')
print(skew(df_listings['availability_365']))
print('\n Asimetria de price:')
print(skew(df_listings['price']))

Asimetria de availability_365 es:
-0.05738325463519655

 Asimetria de price:
0.8369389713897533


Como una de las variables se sale considerablemente de los limites de la normalidad, realizamos test no paramétricos para comprobar nuestra hipótesis.

In [168]:
# MANN WHITNEY


from scipy.stats import mannwhitneyu
stat , p_valor = mannwhitneyu(df_listings['availability_365'], df_listings['price'])

# Mostramos resultado
print(f"El p-valor obtenido en el test de Mann Whitney de {p_valor}")

El p-valor obtenido en el test de Mann Whitney de 2.590793409747073e-119


##### De acuerdo al p-valor del test, menor que el nivel de significaición, rechazamos la hipotesis nula, y existe una diferencia significativa entre la aceptación del alquiler de acuerdo al precio

In [169]:
top_200.columns

Index(['id', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed',
       'property_type', 'room_type', 'latitude', 'longitude', 'price',
       'minimum_nights', 'maximum_nights', 'accommodates', 'bedrooms', 'beds',
       'number_of_reviews', 'number_of_reviews_ltm', 'reviews_per_month',
       'review_scores_rating', 'review_scores_cleanliness',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'host_is_superhost', 'host_response_rate',
       'host_listings_count', 'availability_365', 'price_normalized',
       'weighted_score'],
      dtype='object')

## **CONTRASTE 7**
1. **Hipotesis nula (Ho)**: no existe una correlación entre la ocupación y las reseñas de limpieza.

2. **Hipotesis alternativa (H1)**: existe una correlación entre la ocupación y las reseñas de limpieza.

### CONTRASTE 7 - TOP_200

In [170]:
# Comprobamos la normalidad de nuestros datos
import scipy.stats as stats

stat1, p_norm1 = stats.shapiro(top_200['availability_365'])
stat2, p_norm2 = stats.shapiro(top_200['review_scores_cleanliness'])
print(f"Normalidad Grupo 1: p-value = {p_norm1}")
print(f"Normalidad Grupo 2: p-value = {p_norm2}")

Normalidad Grupo 1: p-value = 7.287925654964056e-06
Normalidad Grupo 2: p-value = 9.537975742546223e-09


In [171]:
# Realizamos un test de spearman para poder determinar nuetra hipótesis. 

from scipy.stats import spearmanr

stats, p_value = spearmanr (top_200['availability_365'],top_200['review_scores_cleanliness']) 

print(f'El p_valor es de: {p_value}')
print(f'El stats es de: {stats}')

El p_valor es de: 0.19831904736118766
El stats es de: -0.09134133027677857


##### Como el p-valor no sale mayor que el nivel de significación, no rechazamos la hipotesis nula, y de acuerdo a los datos, no existe una correlación entre la ocupación y las reseñas de limpieza.

### CONTRASTE 7 - BOTTOM_200

In [172]:
# Comprobamos la normalidad de nuestros datos
import scipy.stats as stats

stat1, p_norm1 = stats.shapiro(bottom_200['availability_365'])
stat2, p_norm2 = stats.shapiro(bottom_200['review_scores_cleanliness'])
print(f"Normalidad Grupo 1: p-value = {p_norm1}")
print(f"Normalidad Grupo 2: p-value = {p_norm2}")

Normalidad Grupo 1: p-value = 1.0032024516659632e-10
Normalidad Grupo 2: p-value = 5.209081777035607e-11


In [173]:
# Realizamos un test de spearman para poder determinar nuetra hipótesis. 

from scipy.stats import spearmanr

stats, p_value = spearmanr (bottom_200['availability_365'],bottom_200['review_scores_cleanliness']) 

print(f'El p_valor es de: {p_value}')
print(f'El stats es de: {stats}')

El p_valor es de: 0.005084580763176932
El stats es de: -0.19738922406306852


##### Como el p-valor no sale menor que el nivel de significación, rechazamos la hipotesis nula, y de acuerdo a los datos, existe una correlación entre la ocupación y las reseñas de limpieza.

### CONTRASTE 7 - DF_LISTINGS

In [174]:
# Comprobamos la asimetria de los datos con 'skew'

print('Asimetria de availability_365 es:')
print(skew(top_200['availability_365']))
print('\n Asimetria de review_scores_cleanliness:')
print(skew(top_200['review_scores_cleanliness']))

Asimetria de availability_365 es:
-0.4217486256345294

 Asimetria de review_scores_cleanliness:
-0.639417820842896


In [176]:
# Comprobamos la normalidad de nuestros datos
import scipy.stats as stats

stat1, p_norm1 = stats.shapiro(df_listings['availability_365'])
stat2, p_norm2 = stats.shapiro(df_listings['review_scores_cleanliness'])
print(f"Normalidad Grupo 1: p-value = {p_norm1}")
print(f"Normalidad Grupo 2: p-value = {p_norm2}")

Normalidad Grupo 1: p-value = 0.0
Normalidad Grupo 2: p-value = 0.0


/Users/juliobrionesmorales/Documents/1. FORMACION/3. DATA ANALYTICS /PROYECTOS_BOOTCAMP/PROYECTO FINAL 2/entorno/lib/python3.10/site-packages/scipy/stats/_morestats.py:1882: UserWarning:

p-value may not be accurate for N > 5000.



Como de acuerdo al test de shapiro y a la asimetria con skew podemos determinar que no siguen una distribución normal, y por lo tanto tenemos que realizar un test no paramétrico determinar nuestra hipotesis.

In [177]:
# Realizamos un test de spearman para poder determinar nuetra hipótesis. 

from scipy.stats import spearmanr

stats, p_value = spearmanr (df_listings['availability_365'],df_listings['review_scores_cleanliness']) 

print(f'El p_valor es de: {p_value}')
print(f'El stats es de: {stats}')

El p_valor es de: 5.0872589523558944e-30
El stats es de: -0.0835054054133427


##### Como el p-valor no sale menor que el nivel de significación, rechazamos la hipotesis nula, y de acuerdo a los datos, existe una correlación entre la ocupación y las reseñas de limpieza.

## **CONTRASTE 8**
1. **Hipotesis nula (Ho)**: no existe una correlación entre la ocupación y las reseñas de comunicación.

2. **Hipotesis alternativa (H1)**: existe una correlación entre la ocupación y las reseñas de comunicación.

### CONTRASTE 8 - TOP_200

In [179]:
# Comprobamos la normalidad de nuestros datos
import scipy.stats as stats

stat1, p_norm1 = stats.shapiro(top_200['availability_365'])
stat2, p_norm2 = stats.shapiro(top_200['review_scores_communication'])
print(f"Normalidad Grupo 1: p-value = {p_norm1}")
print(f"Normalidad Grupo 2: p-value = {p_norm2}")

Normalidad Grupo 1: p-value = 7.287925654964056e-06
Normalidad Grupo 2: p-value = 9.198982686697832e-10


In [180]:
# Realizamos un test de spearman para poder determinar nuetra hipótesis. 

from scipy.stats import spearmanr

stats, p_value = spearmanr (top_200['availability_365'],top_200['review_scores_communication']) 

print(f'El p_valor es de: {p_value}')
print(f'El stats es de: {stats}')

El p_valor es de: 0.03835295293121079
El stats es de: -0.14657309463825136


##### Como el p-valor no sale menor que el nivel de significación, rechazamos la hipotesis nula, y de acuerdo a los datos, existe una correlación entre la ocupación y las reseñas de comunicación.

### CONTRASTE 8 - BOTTOM_200

In [181]:
# Comprobamos la normalidad de nuestros datos
import scipy.stats as stats

stat1, p_norm1 = stats.shapiro(bottom_200['availability_365'])
stat2, p_norm2 = stats.shapiro(bottom_200['review_scores_communication'])
print(f"Normalidad Grupo 1: p-value = {p_norm1}")
print(f"Normalidad Grupo 2: p-value = {p_norm2}")

Normalidad Grupo 1: p-value = 1.0032024516659632e-10
Normalidad Grupo 2: p-value = 2.0240961684514502e-12


In [182]:
# Realizamos un test de spearman para poder determinar nuetra hipótesis. 

from scipy.stats import spearmanr

stats, p_value = spearmanr (bottom_200['availability_365'],bottom_200['review_scores_communication']) 

print(f'El p_valor es de: {p_value}')
print(f'El stats es de: {stats}')

El p_valor es de: 0.012322704505660568
El stats es de: -0.1766852002100979


##### Como el p-valor no sale menor que el nivel de significación, rechazamos la hipotesis nula, y de acuerdo a los datos, existe una correlación entre la ocupación y las reseñas de comunicación.

### CONTRASTE 8 - DF_LISTINGS

In [183]:
# Comprobamos la asimetria de los datos con 'skew'

print('Asimetria de availability_365 es:')
print(skew(top_200['availability_365']))
print('\n Asimetria de review_scores_communication:')
print(skew(top_200['review_scores_communication']))

Asimetria de availability_365 es:
-0.4217486256345294

 Asimetria de review_scores_communication:
-0.5731740003193707


In [184]:
# Comprobamos la normalidad de nuestros datos
import scipy.stats as stats

stat1, p_norm1 = stats.shapiro(df_listings['availability_365'])
stat2, p_norm2 = stats.shapiro(df_listings['review_scores_communication'])
print(f"Normalidad Grupo 1: p-value = {p_norm1}")
print(f"Normalidad Grupo 2: p-value = {p_norm2}")

Normalidad Grupo 1: p-value = 0.0
Normalidad Grupo 2: p-value = 0.0


/Users/juliobrionesmorales/Documents/1. FORMACION/3. DATA ANALYTICS /PROYECTOS_BOOTCAMP/PROYECTO FINAL 2/entorno/lib/python3.10/site-packages/scipy/stats/_morestats.py:1882: UserWarning:

p-value may not be accurate for N > 5000.



In [185]:
# Como la asimetria se asemeja a una distribución normal, vamos a realizar un test de pearson

from scipy.stats import pearsonr

stats, p_value = pearsonr (df_listings['availability_365'],df_listings['review_scores_communication']) 

print(f'El p_valor es de: {p_value}')
print(f'El stats es de: {stats}')

El p_valor es de: 8.836279492880494e-80
El stats es de: -0.13832395418823576


In [186]:
from scipy.stats import spearmanr

stats, p_value = spearmanr (df_listings['availability_365'],df_listings['review_scores_communication']) 

print(f'El p_valor es de: {p_value}')
print(f'El stats es de: {stats}')

El p_valor es de: 2.087053956363605e-80
El stats es de: -0.13887394638466624


##### Como el p-valor no sale menor que el nivel de significación, rechazamos la hipotesis nula, y de acuerdo a los datos, existe una correlación entre la ocupación y las reseñas de comunicación.